# HSE 2022: Mathematical Methods for Data Analysis

## Homework 2

In [188]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn import datasets
from sklearn.datasets import load_boston
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLSResults
from math import sqrt
import random
import sys

import warnings

warnings.filterwarnings("ignore")

% matplotlib inline

sns.set(style="darkgrid")

UsageError: Line magic function `%` not found.


### Data

For this homework we use Dataset from seaborn on diamonds prices.

In [189]:
data = sns.load_dataset('diamonds')
y = data.price
X = data.drop(['price'], axis=1)
columns = data.drop(['price'], axis=1).columns

## Linear regression

#### 0. [0.25 points] Encode categorical variables.

In [190]:
categories = (data.dtypes == "category")
object_cols = list(categories[categories].index)

encoded_data = X.copy()
label_encoder = LabelEncoder()
for col in object_cols:
    encoded_data[col] = label_encoder.fit_transform(encoded_data[col])

#### 1. [0.25 points] Split the data into train and test sets with ratio 80:20 with random_state=17.

In [191]:
X_train, X_test, y_train, y_test = train_test_split(encoded_data, y, test_size=0.2, random_state=17)

#### 2. [1 point] Train models on train data using StatsModels library and apply it to the test set; use $RMSE$ and $R^2$ as the quality measure.

* [`LinearRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html);
* [`Ridge`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) with $\alpha = 0.01$;
* [`Lasso`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) with $\alpha = 0.01$
* [`ElasticNet`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html) with $\alpha = 0.01$, $l_{1}$_$ratio = 0.6$

Don't forget to scale the data before training the models with StandardScaler!

In [192]:
# scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [193]:
# for statsmodels
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)
model = sm.OLS(y_train, X_train)

# Linear Regression from statsmodels
results_lr = model.fit()
y_test_predicted = results_lr.predict(X_test)
y_train_predicted = results_lr.predict(X_train)

print("Train RMSE = %.4f" % mean_squared_error(y_train, y_train_predicted, squared=False))
print("Test RMSE = %.4f" % mean_squared_error(y_test, y_test_predicted, squared=False))
print("Train R2 = %.4f" % r2_score(y_train, y_train_predicted))
print("Test R2 = %.4f" % r2_score(y_test, y_test_predicted))

Train RMSE = 1347.9933
Test RMSE = 1370.9682
Train R2 = 0.8853
Test R2 = 0.8839


In [194]:
# Ridge from statsmodels
results_ridge = model.fit_regularized(L1_wt=0, alpha=0.01)
y_test_predicted = results_ridge.predict(X_test)
y_train_predicted = results_ridge.predict(X_train)

print("Train RMSE = %.4f" % mean_squared_error(y_train, y_train_predicted, squared=False))
print("Test RMSE = %.4f" % mean_squared_error(y_test, y_test_predicted, squared=False))
print("Train R2 = %.4f" % r2_score(y_train, y_train_predicted))
print("Test R2 = %.4f" % r2_score(y_test, y_test_predicted))

Train RMSE = 1365.9920
Test RMSE = 1383.9541
Train R2 = 0.8822
Test R2 = 0.8817


In [195]:
# Lasso from statsmodels
results_lasso = model.fit_regularized(L1_wt=1, alpha=0.01)
y_test_predicted = results_lasso.predict(X_test)
y_train_predicted = results_lasso.predict(X_train)

print("Train RMSE = %.4f" % mean_squared_error(y_train, y_train_predicted, squared=False))
print("Test RMSE = %.4f" % mean_squared_error(y_test, y_test_predicted, squared=False))
print("Train R2 = %.4f" % r2_score(y_train, y_train_predicted))
print("Test R2 = %.4f" % r2_score(y_test, y_test_predicted))

Train RMSE = 1348.6037
Test RMSE = 1370.0240
Train R2 = 0.8852
Test R2 = 0.8841


In [196]:
# Elastic net from statsmodels
results_elastic = model.fit_regularized(L1_wt=0.6, alpha=0.01)
y_test_predicted = results_elastic.predict(X_test)
y_train_predicted = results_elastic.predict(X_train)

print("Train RMSE = %.4f" % mean_squared_error(y_train, y_train_predicted, squared=False))
print("Test RMSE = %.4f" % mean_squared_error(y_test, y_test_predicted, squared=False))
print("Train R2 = %.4f" % r2_score(y_train, y_train_predicted))
print("Test R2 = %.4f" % r2_score(y_test, y_test_predicted))

Train RMSE = 1353.6044
Test RMSE = 1372.8616
Train R2 = 0.8844
Test R2 = 0.8836


The best quality has Linear Regression model because it has the smallest RMSE and the highest R2 score

#### 3. [1 point] Explore the values of the parameters of the resulting models and compare the number of zero weights in them. Comment on the significance of the coefficients, overal model significance and other related factors from the results table

In [197]:
features = list(X.iloc[:, :10].columns)
features.insert(0, "const")

# linear regression
results_lr.summary2(xname=features)

<class 'statsmodels.iolib.summary2.Summary'>
"""
                  Results: Ordinary least squares
====================================================================
Model:              OLS              Adj. R-squared:     0.885      
Dependent Variable: price            AIC:                744418.8263
Date:               2022-10-16 00:46 BIC:                744505.5512
No. Observations:   43152            Log-Likelihood:     -3.7220e+05
Df Model:           9                F-statistic:        3.701e+04  
Df Residuals:       43142            Prob (F-statistic): 0.00       
R-squared:          0.885            Scale:              1.8175e+06 
--------------------------------------------------------------------
             Coef.    Std.Err.    t     P>|t|    [0.025     0.975]  
--------------------------------------------------------------------
const       3928.6813   6.4899 605.3537 0.0000  3915.9610  3941.4016
carat       5257.1453  31.0710 169.1979 0.0000  5196.2456  5318.0450
cut           76.4610   6.6590  11.4824 0.0000    63.4093    89.5128
color       -455.4350   6.8100 -66.8778 0.0000  -468.7826  -442.0874
clarity      491.4240   6.7022  73.3231 0.0000   478.2876   504.5604
depth       -226.2704   7.9533 -28.4498 0.0000  -241.8590  -210.6818
table       -213.2612   6.9923 -30.4996 0.0000  -226.9662  -199.5562
x          -1383.2878  48.3537 -28.6077 0.0000 -1478.0619 -1288.5137
y             42.1665  29.5137   1.4287 0.1531   -15.6809   100.0138
z              3.3540  29.6459   0.1131 0.9099   -54.7524    61.4604
--------------------------------------------------------------------
Omnibus:            11265.146      Durbin-Watson:         1.999     
Prob(Omnibus):      0.000          Jarque-Bera (JB):      360275.892
Skew:               0.611          Prob(JB):              0.000     
Kurtosis:           17.103         Condition No.:         18        
====================================================================

"""

The values of the features are in "coef" column. There is no zero weights.
Linear Regression summary:
    We consider significance level = 0.05.
    1. P-values of "y" is higher than 0.05, so we can call "y" coefficient insignificant
    2. P-values of "z" is higher than 0.05, so we can call "z" coefficient insignificant
    3. R2_adj show that ~88.5% of the variance is explained, so the model is valuable
    4. As F-statistics ia a pretty large number, we can say that the model is significant
    5. Durbin-Watson value is nearly 2, so there is no autocorrelation
    6. Large Jarque-Bera value indicates that errors are not normally distributed

In [198]:
# ridge
OLSResults(model, results_ridge.params, model.normalized_cov_params).summary2(xname=features)

<class 'statsmodels.iolib.summary2.Summary'>
"""
                  Results: Ordinary least squares
====================================================================
Model:              OLS              Adj. R-squared:     0.882      
Dependent Variable: price            AIC:                745563.5541
Date:               2022-10-16 00:46 BIC:                745650.2790
No. Observations:   43152            Log-Likelihood:     -3.7277e+05
Df Model:           9                F-statistic:        3.591e+04  
Df Residuals:       43142            Prob (F-statistic): 0.00       
R-squared:          0.882            Scale:              1.8664e+06 
---------------------------------------------------------------------
            Coef.    Std.Err.     t      P>|t|     [0.025     0.975] 
---------------------------------------------------------------------
const     3889.7835    6.5765  591.4627  0.0000  3876.8933  3902.6736
carat     4219.7372   31.4858  134.0201  0.0000  4158.0243  4281.4500
cut         81.8556    6.7479   12.1306  0.0000    68.6296    95.0816
color     -424.7483    6.9009  -61.5498  0.0000  -438.2742  -411.2225
clarity    496.6610    6.7917   73.1280  0.0000   483.3492   509.9728
depth     -162.6547    8.0595  -20.1817  0.0000  -178.4514  -146.8579
table     -203.6562    7.0856  -28.7421  0.0000  -217.5442  -189.7682
x         -312.9137   48.9993   -6.3861  0.0000  -408.9533  -216.8741
y           21.4162   29.9077    0.7161  0.4739   -37.2035    80.0359
z          -39.9802   30.0417   -1.3308  0.1833   -98.8625    18.9021
--------------------------------------------------------------------
Omnibus:            13066.102      Durbin-Watson:         1.998     
Prob(Omnibus):      0.000          Jarque-Bera (JB):      183804.270
Skew:               1.067          Prob(JB):              0.000     
Kurtosis:           12.883         Condition No.:         18        
====================================================================

"""

The values of the features are in "coef" column. There is no zero weights.
Ridge Regression summary:
    We consider significance level = 0.05.
    1. P-values of "y" is higher than 0.05, so we can call "y" coefficient insignificant
    2. P-values of "z" is higher than 0.05, so we can call "z" coefficient insignificant
    3. R2_adj show that ~88.2% of the variance is explained, so the model is valuable
    4. As F-statistics ia a pretty large number, we can say that the model is significant
    5. Durbin-Watson value is nearly 2, so there is no autocorrelation
    6. Large Jarque-Bera value indicates that errors are not normally distributed

In [199]:
# lasso
OLSResults(model, results_lasso.params, model.normalized_cov_params).summary2(xname=features)

<class 'statsmodels.iolib.summary2.Summary'>
"""
                  Results: Ordinary least squares
====================================================================
Model:              OLS              Adj. R-squared:     0.885      
Dependent Variable: price            AIC:                744457.9003
Date:               2022-10-16 00:46 BIC:                744544.6252
No. Observations:   43152            Log-Likelihood:     -3.7222e+05
Df Model:           9                F-statistic:        3.697e+04  
Df Residuals:       43142            Prob (F-statistic): 0.00       
R-squared:          0.885            Scale:              1.8192e+06 
--------------------------------------------------------------------
             Coef.    Std.Err.    t     P>|t|    [0.025     0.975]  
--------------------------------------------------------------------
const       3928.6713   6.4928 605.0782 0.0000  3915.9452  3941.3974
carat       5063.8698  31.0850 162.9037 0.0000  5002.9425  5124.7970
cut           77.5642   6.6620  11.6428 0.0000    64.5066    90.6218
color       -453.1266   6.8130 -66.5087 0.0000  -466.4802  -439.7729
clarity      495.2762   6.7052  73.8644 0.0000   482.1339   508.4186
depth       -214.9442   7.9569 -27.0135 0.0000  -230.5399  -199.3485
table       -213.4343   6.9954 -30.5105 0.0000  -227.1455  -199.7231
x          -1201.4586  48.3756 -24.8361 0.0000 -1296.2757 -1106.6416
y             54.7441  29.5270   1.8540 0.0637    -3.1294   112.6176
z             -1.4625  29.6593  -0.0493 0.9607   -59.5953    56.6702
--------------------------------------------------------------------
Omnibus:            11656.263      Durbin-Watson:         1.999     
Prob(Omnibus):      0.000          Jarque-Bera (JB):      315053.588
Skew:               0.713          Prob(JB):              0.000     
Kurtosis:           16.160         Condition No.:         18        
====================================================================

"""

The values of the features are in "coef" column. There is no zero weights.
Lasso Regression summary:
    We consider significance level = 0.05.
    1. P-values of "y" is higher than 0.05, so we can call "y" coefficient insignificant
    2. P-values of "z" is higher than 0.05, so we can call "z" coefficient insignificant
    3. R2_adj show that ~88.5% of the variance is explained, so the model is valuable
    4. As F-statistics ia a pretty large number, we can say that the model is significant
    5. Durbin-Watson value is nearly 2, so there is no autocorrelation
    6. Large Jarque-Bera value indicates that errors are not normally distributed

In [200]:
# elastic
OLSResults(model, results_elastic.params, model.normalized_cov_params).summary2(xname=features)

<class 'statsmodels.iolib.summary2.Summary'>
"""
                  Results: Ordinary least squares
====================================================================
Model:              OLS              Adj. R-squared:     0.884      
Dependent Variable: price            AIC:                744777.3304
Date:               2022-10-16 00:46 BIC:                744864.0552
No. Observations:   43152            Log-Likelihood:     -3.7238e+05
Df Model:           9                F-statistic:        3.666e+04  
Df Residuals:       43142            Prob (F-statistic): 0.00       
R-squared:          0.884            Scale:              1.8327e+06 
---------------------------------------------------------------------
            Coef.    Std.Err.     t      P>|t|     [0.025     0.975] 
---------------------------------------------------------------------
const     3913.0232    6.5169  600.4416  0.0000  3900.2499  3925.7965
carat     4675.8946   31.2003  149.8669  0.0000  4614.7414  4737.0478
cut         79.7319    6.6867   11.9240  0.0000    66.6258    92.8379
color     -440.5288    6.8383  -64.4208  0.0000  -453.9320  -427.1256
clarity    496.4432    6.7301   73.7649  0.0000   483.2521   509.6342
depth     -190.1759    7.9864  -23.8124  0.0000  -205.8294  -174.5223
table     -209.3948    7.0214  -29.8225  0.0000  -223.1568  -195.6328
x         -766.0692   48.5550  -15.7774  0.0000  -861.2378  -670.9005
y           20.1115   29.6365    0.6786  0.4974   -37.9767    78.1996
z          -26.8075   29.7693   -0.9005  0.3679   -85.1558    31.5408
--------------------------------------------------------------------
Omnibus:            12406.161      Durbin-Watson:         1.999     
Prob(Omnibus):      0.000          Jarque-Bera (JB):      244618.648
Skew:               0.894          Prob(JB):              0.000     
Kurtosis:           14.526         Condition No.:         18        
====================================================================

"""

The values of the features are in "coef" column. There is no zero weights.
Elastic Net Regression summary:
    We consider significance level = 0.05.
    1. P-values of "y" is higher than 0.05, so we can call "y" coefficient insignificant
    2. P-values of "z" is higher than 0.05, so we can call "z" coefficient insignificant
    3. R2_adj show that ~88.4% of the variance is explained, so the model is valuable
    4. As F-statistics ia a pretty large number, we can say that the model is significant
    5. Durbin-Watson value is nearly 2, so there is no autocorrelation
    6. Large Jarque-Bera value indicates that errors are not normally distributed

<bold> Summary: <bold>
    1. As Linear Regression model has the smallest AIC value, it should be selected.
    2. Linear Regression model also has the smallest BIC value
    3. In all models "y" and "z" are insignificant
    4. The Ridge model is the most skewed

#### 4. [1 point] Implement one of the elimination algorithms that were described in the Seminar_4 (Elimination by P-value, Forward elimination, Backward elimination), make conclusions.

<bold>We consider significance level = 0.05.<bold>

In [238]:
# p-value elimination for linear regression model
model = sm.OLS(y_train, X_train)
results = model.fit()
sign_level = 0.05

features = list(X.iloc[:, :10].columns)
features.insert(0, "const")
features = np.array(features)

print("Start from model with all features:")
print("R2_adj = %.10f" % results.rsquared_adj)
print("RMSE = %.10f" % mean_squared_error(y_train, results.predict(X_train), squared=False))
print("AIC = %.10f\n" % results.aic)
p_values = np.array(results.pvalues)
X_eliminated = X_train

while np.max(p_values) > sign_level:
    insign_feature_index = np.argmax(p_values)
    X_eliminated = np.delete(X_eliminated, insign_feature_index, axis=1)
    print(f'Feature \"{features[insign_feature_index]}\" was eliminated.')
    features = np.delete(features, insign_feature_index)
    model = sm.OLS(y_train, X_eliminated)
    results = model.fit()
    p_values = np.array(results.pvalues)
    print("Current:\nR2_adj = %.10f" % results.rsquared_adj)
    print("RMSE = %.10f" % mean_squared_error(y_train, results.predict(X_eliminated), squared=False))
    print("AIC = %.10f\n" % results.aic)

print("Features after p-value elimination algorithm:")
print(features)

Start from model with all features:
R2_adj = 0.8853021693
RMSE = 1347.9932745208
AIC = 744418.8263415196

Feature "z" was eliminated.
Current:
R2_adj = 0.8853047938
RMSE = 1347.9934744846
AIC = 744416.8391440106

Feature "y" was eliminated.
Current:
R2_adj = 0.8853018368
RMSE = 1348.0264734788
AIC = 744416.9518473670

Features after p-value elimination algorithm:
['const' 'carat' 'cut' 'color' 'clarity' 'depth' 'table' 'x']


Summary: the elimination of the "z" increased model's quality, so we can remove it.

#### 5. [1 point] Find the best (in terms of RMSE) $\alpha$ for Lasso regression using cross-validation with 4 folds. You must select values from range $[10^{-4}, 10^{3}]$.

In [ ]:
parameters = {'alpha': np.logspace(-4, 3)}
search = GridSearchCV(Lasso(), parameters, cv=4, scoring='neg_root_mean_squared_error')
search.fit(X_train, y_train)
print(search.best_params_)

## Gradient descent

#### 6. [3.5 points] Implement a Ridge regression model for the MSE loss function, trained by gradient descent.

All calculations must be vectorized, and python loops can only be used for gradient descent iterations. As a stop criterion, you must use (simultaneously):

* checking for the Absolute-value norm of the weight difference on two adjacent iterations (for example, less than some small number of the order of $10^{-6}$, set by the `tolerance` parameter);
* reaching the maximum number of iterations (for example, 10000, set by the `max_iter` parameter).

You need to implement:

* Full gradient descent:

$$
w_{k + 1} = w_{k} - \eta_{k} \nabla_{w} Q(w_{k}).
$$

* Stochastic Gradient Descent:

$$
w_{k + 1} = w_{k} - \eta_{k} \nabla_{w} q_{i_{k}}(w_{k}).
$$

$\nabla_{w} q_{i_{k}}(w_{k}) \, $ is the estimate of the gradient over the batch of objects selected randomly.

* Momentum method:

$$
h_0 = 0, \\
h_{k + 1} = \alpha h_{k} + \eta_k \nabla_{w} Q(w_{k}), \\
w_{k + 1} = w_{k} - h_{k + 1}.
$$

* Adagrad method:

$$
G_0 = 0, \\
G_{k + 1} = G_{k} + (\nabla_{w} Q(w_{k+1}))^2, \\
w_{k + 1} = w_{k} - \eta * \frac{\nabla_{w} Q(w_{k+1})}{\sqrt{G_{k+1} + \epsilon}}.
$$



To make sure that the optimization process really converges, we will use the `loss_history` class attribute. After calling the `fit` method, it should contain the values of the loss function for all iterations, starting from the first one (before the first step on the anti-gradient).

You need to initialize the weights with a random vector from normal distribution. The following is a template class that needs to contain the code implementing all variations of the models.

In [239]:
from sklearn.base import BaseEstimator
from numpy import linalg as la


class LinReg(BaseEstimator):
    def __init__(self, delta=1.0, gd_type='Momentum',
                 tolerance=1e-4, max_iter=1000, w0=None, eta=1e-2, alpha=1e-3, reg_cf=1.0, epsilon=10e-8):
        """
        gd_type: str
            'GradientDescent', 'StochasticDescent', 'Momentum', 'Adagrad'
        delta: float
            proportion of object in a batch (for stochastic GD)
        tolerance: float
            for stopping gradient descent
        max_iter: int
            maximum number of steps in gradient descent
        w0: np.array of shape (d)
            init weights
        eta: float
            learning rate
        alpha: float
            momentum coefficient
        reg_cf: float
            regularization coefficient
        epsilon: float
            numerical stability
        """

        self.delta = delta
        self.gd_type = gd_type
        self.tolerance = tolerance
        self.max_iter = max_iter
        self.w0 = w0
        self.alpha = alpha
        self.w = None
        self.eta = eta
        self.loss_history = None  # list of loss function values at each training iteration
        self.epsilon = epsilon
        self.reg_cf = reg_cf

    def fit(self, X, y):
        """
        X: np.array of shape (l, d)
        y: np.array of shape (l)
        ---
        output: self
        """
        self.loss_history = []
        X = np.insert(X, 0, 1, axis=1)
        if self.gd_type == 'GradientDescent':
            self.w = self.full_grad_descent_calc(X, y)
        if self.gd_type == 'Momentum':
            self.w = self.momentum_descent_calc(X, y)
        if self.gd_type == 'Adagrad':
            self.w = self.adagrad_descent_calc(X, y)

        return self

    def predict(self, X):
        if self.w is None:
            raise Exception('Not trained yet')
        return np.dot(np.insert(X, 0, 1, axis=1), self.w)

    def calc_gradient(self, X, y, w):
        """
        X: np.array of shape (l, d) (l can be equal to 1 if stochastic)
        y: np.array of shape (l)
        ---
        output: np.array of shape (d)
        """
        return 2 * np.dot(X.T, np.dot(X, w) - y) / np.shape(X)[0] + 2 * self.reg_cf * w / np.shape(X)[0]

    def full_grad_descent_calc(self, X, y):
        w = np.random.normal(0, 1, np.shape(X)[1])
        self.w = w.copy()
        iter_num = 0
        while iter_num < self.max_iter and abs(la.norm(w - self.w) - self.tolerance) > 10e-6:
                #and abs(la.norm(w - self.w) - self.tolerance) > 10e-6:
            gradient = self.calc_gradient(X, y, w)
            self.loss_history = np.append(self.loss_history, self.calc_loss(X, y))
            self.w = w
            w -= self.eta * gradient
            iter_num += 1
        return w

    def momentum_descent_calc(self, X, y):
        w = np.random.normal(0, 1, np.shape(X)[1])
        self.w = w.copy()
        iter_num = 0
        h = np.zeros(np.shape(X)[1])
        while iter_num < self.max_iter and abs(la.norm(w - self.w) - self.tolerance) > 0:
            gradient = self.calc_gradient(X, y, w)
            self.loss_history = np.append(self.loss_history, self.calc_loss(X, y))
            h = self.alpha * h + self.eta * gradient
            self.w = w
            w -= h
            iter_num += 1
        return w

    def adagrad_descent_calc(self, X, y):
        w = np.random.normal(0, 1, np.shape(X)[1])
        self.w = w.copy()
        iter_num = 0
        g = np.zeros(np.shape(X)[1])
        while iter_num < self.max_iter and abs(la.norm(w - self.w) - self.tolerance) > 10e-6:
            gradient = self.calc_gradient(X, y, w)
            np.append(self.loss_history, self.calc_loss(X, y))
            g += np.square(gradient)
            self.w = w
            w -= self.eta * np.divide(gradient, (np.sqrt(g + self.epsilon)))
            iter_num += 1
        return w

    # def stochastic_grad_descent_calc(self, X, y):
    #     w = np.random.normal(0, 1, np.shape(X)[1])
    #     w_prev = w.copy()
    #     iter_num = 0
    #     while iter_num < self.max_iter and abs(w - w_prev) < self.tolerance:
    #
    #     return w

    def calc_loss(self, X, y):
        """
        X: np.array of shape (l, d)
        y: np.array of shape (l)
        ---
        output: float 
        """
        return np.sum(np.square(np.dot(X, self.w) - y)) / np.shape(X)[0] + self.reg_cf * np.sum(np.square(self.w))

#### 7. [1 points] Train and validate "hand-written" models on the same data, and compare the quality with the Sklearn or StatsModels methods. Investigate the effect of the `max_iter` and `alpha` parameters on the optimization process. Is it consistent with your expectations?

In [ ]:
# TODO условие по выходу из цикла в модели
# your code here
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
X_train, X_test, y_train, y_test = train_test_split(encoded_data, y, test_size=0.2, random_state=17)
# scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

# Full gradient descent
model = LinReg(gd_type='GradientDescent')
model.fit(X_train, y_train)
y_train_predicted = model.predict(X_train)
y_test_predicted = model.predict(X_test)
print("Full gradient descent: ")
print("Train RMSE = %.4f" % mean_squared_error(y_train, y_train_predicted, squared=False))
print("Train R2 = %.4f" % r2_score(y_train, y_train_predicted))
print("Train RMSE = %.4f" % mean_squared_error(y_test, y_test_predicted, squared=False))
print("Train R2 = %.4f" % r2_score(y_test, y_test_predicted))

In [240]:
model = LinReg(gd_type='Momentum')
model.fit(X_train, y_train)
y_train_predicted = model.predict(X_train)
y_test_predicted = model.predict(X_test)
print("Momentum: ")
print("Train RMSE = %.4f" % mean_squared_error(y_train, y_train_predicted, squared=False))
print("Train R2 = %.4f" % r2_score(y_train, y_train_predicted))
print("Train RMSE = %.4f" % mean_squared_error(y_test, y_test_predicted, squared=False))
print("Train R2 = %.4f" % r2_score(y_test, y_test_predicted))
print(model.loss_history)

Momentum: 
Train RMSE = 1418.1395
Train R2 = 0.8731
Train RMSE = 1436.6256
Train R2 = 0.8725
[31272297.98160338 28163401.21300612 25501876.94114036 23226105.28166647
 21280493.71949341 19617702.09419871 18197389.29261866 16985150.62769159
 15951619.23681988 15071705.95374813 14323956.10306663 13690005.03779136
 13154117.08304106 12702794.94566768 12324448.67079219 12009114.93054645
 11748218.86765017 11534371.92866996 11361200.14424394 11223198.17598767
 11115605.17732585 11034299.12927676 10975706.83008922 10936727.15462852
 10914665.56820312 10907178.19080239 10912223.97048477 10928023.74650928
 10953025.17014413 10985872.60931228 11025381.29689624 11070515.09546212
 11120367.3466111  11174144.35385483 11231151.11614158 11290778.98687276
 11352494.98208284 11415832.50279027 11480383.27152789 11545790.31271232
 11611741.83164078 11677965.86821276 11744225.6195511  11810315.34104389
 11876056.74836317 11941295.85409638 12005900.18205076 12069756.31031337
 12132767.70098437 12194852.780

In [ ]:
# model = LinReg(gd_type='Stochastic')
# model.fit(X_train, y_train)
# y_train_predicted = model.predict(X_train)
# y_test_predicted = model.predict(X_test)
# print("Stochastic: ")
# print("Train RMSE = %.4f" % mean_squared_error(y_train, y_train_predicted, squared=False))
# print("Train R2 = %.4f" % r2_score(y_train, y_train_predicted))
# print("Train RMSE = %.4f" % mean_squared_error(y_test, y_test_predicted, squared=False))
# print("Train R2 = %.4f" % r2_score(y_test, y_test_predicted))

In [ ]:
# model = LinReg(gd_type='Adagrad')
# model.fit(X_train, y_train)
# y_train_predicted = model.predict(X_train)
# y_test_predicted = model.predict(X_test)
# print("Adagrad: ")
# print("Train RMSE = %.4f" % mean_squared_error(y_train, y_train_predicted, squared=False))
# print("Train R2 = %.4f" % r2_score(y_train, y_train_predicted))
# print("Train RMSE = %.4f" % mean_squared_error(y_test, y_test_predicted, squared=False))
# print("Train R2 = %.4f" % r2_score(y_test, y_test_predicted))

In [ ]:
model = Ridge()
model.fit(X_train, y_train)
y_train_predicted = model.predict(X_train)
y_test_predicted = model.predict(X_test)
print("Ridge from Sklearn: ")
print("Train RMSE = %.4f" % mean_squared_error(y_train, y_train_predicted, squared=False))
print("Train R2 = %.4f" % r2_score(y_train, y_train_predicted))
print("Train RMSE = %.4f" % mean_squared_error(y_test, y_test_predicted, squared=False))
print("Train R2 = %.4f" % r2_score(y_test, y_test_predicted))

#### 8. [1 points] Plot graphs (on the same picture) of the dependence of the loss function value on the iteration number for Full GD, SGD, Momentum and Adagrad. Draw conclusions about the rate of convergence of various modifications of gradient descent.

Don't forget about what *beautiful* graphics should look like!

In [ ]:
# your code here 
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ